In [1]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

FNAME = '../Nerlove1963.dta'
df = pandas.read_stata(FNAME)

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
print list(df)

['cost', 'output', 'Plabor', 'Pfuel', 'Pcapital', 'lc', 'lq', 'lpl', 'lpf', 'lpc']


In [15]:
#part a, general model
y = pandas.DataFrame()
y['lc'] = df['lc']
X1 = df[['lq','lpl','lpf','lpc']]
X1 = sm.add_constant(X1)
model1 = sm.GLS(y,X1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                     lc   R-squared:                       0.926
Model:                            GLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     437.7
Date:                Wed, 21 Mar 2018   Prob (F-statistic):           4.82e-78
Time:                        20:35:39   Log-Likelihood:                -67.542
No. Observations:                 145   AIC:                             145.1
Df Residuals:                     140   BIC:                             160.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5265      1.774     -1.987      0.049      -7.035      -0.018
lq             0.7204      0.017     41.244      0.000       0.686       0.755
lpl            0.4363      0.291      1.499      0.136      -0.139       1.012
lpf            0.4265      0.100      4.249      0.000       0.228       0.625
lpc           -0.2199      0.339     -0.648      0.518      -0.891       0.451
==============================================================================
Omnibus:                       51.403   Durbin-Watson:                   1.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              175.700
Skew:                           1.303   Prob(JB):                     7.03e-39
Kurtosis:                       7.721   Cond. No.                         506.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
# part b

#the restriction is ensuring constant returns to scale

In [16]:
#part c, impose the restriction b5 = 1 - b3 - b4
#i.e. y - x5 = b0 + b1 x1 + b2 x2 + b3 (x3 -x5) + b4 (x4-x5)
X2 = pandas.DataFrame()
y2 = pandas.DataFrame()
y2['lc-lpf'] = y['lc'].add(-1.0*X1['lpf'])
X2['lq'] = X1['lq']
X2['lpl-lpf'] = X1['lpl'].add(-1.0*X1['lpf'])
X2['lpc-lpf'] = X1['lpc'].add(-1.0*X1['lpf'])
X2 = sm.add_constant(X2)
model2 = sm.GLS(y2,X2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                 lc-lpf   R-squared:                       0.932
Model:                            GLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     640.0
Date:                Wed, 21 Mar 2018   Prob (F-statistic):           6.95e-82
Time:                        20:36:28   Log-Likelihood:                -67.838
No. Observations:                 145   AIC:                             143.7
Df Residuals:                     141   BIC:                             155.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.6908      0.885     -5.301      0.000      -6.440      -2.941
lq             0.7207      0.017     41.334      0.000       0.686       0.755
lpl-lpf        0.5929      0.205      2.898      0.004       0.188       0.997
lpc-lpf       -0.0074      0.191     -0.039      0.969      -0.384       0.370
==============================================================================
Omnibus:                       49.416   Durbin-Watson:                   1.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              166.633
Skew:                           1.252   Prob(JB):                     6.55e-37
Kurtosis:                       7.617   Cond. No.                         215.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [52]:
#part d
print '----Covariance Matrix----'
print model2.cov_HC0

model3 = sm.GLS(y2,X2,weights=model2.cov_HC0).fit()

#python is being tempermental about printing...
print '----Coeficients----'
print model3.params
print '----SE----'
print model3.HC0_se

----Covariance Matrix----
[[ 0.64567611 -0.01363163  0.1251872  -0.11646145]
 [-0.01363163  0.00102454 -0.00128702  0.0015356 ]
 [ 0.1251872  -0.00128702  0.02779562 -0.02311692]
 [-0.11646145  0.0015356  -0.02311692  0.02360139]]
----Coeficients----
const     -4.690789
lq         0.720688
lpl-lpf    0.592910
lpc-lpf   -0.007381
dtype: float64
----SE----
const      0.803540
lq         0.032008
lpl-lpf    0.166720
lpc-lpf    0.153627
dtype: float64


In [27]:
#part e
model1.wald_test(([0,0,1,1,1],1))

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[ 0.57366045]]), p=0.450080906804, df_denom=140, df_num=1>

In [26]:
#part f
model4 = sm.GLS(y,X1,weights=model1.cov_HC0).fit()
model4.wald_test(([0,0,1,1,1],1))

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[ 0.57366045]]), p=0.450080906804, df_denom=140, df_num=1>